In [1]:
from flask import Flask, render_template, request, redirect, url_for, Response
import time
import random
import string
import logging
import logging.handlers
import sqlite3


# template_dir = os.path.abspath('/templates')
app = Flask(__name__)

log = logging.getLogger('flask')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)


@app.route('/')
@app.route('/index/')
def index():
    return render_template('index.html')

app.run(host='localhost', port=63700, use_reloader=False, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:63700
Press CTRL+C to quit
127.0.0.1 - - [14/Jan/2023 18:40:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 18:40:15] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Jan/2023 18:40:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2023 18:40:18] "GET /static/css/style.css HTTP/1.1" 304 -


@app.route('/register_page')
def register1():
    return render_template('register1.html')

def CheckIfCorrectCredentials(loginName,password):

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    params=[loginName,password]
    c.execute("Select * from users where loginName==? and password==?",params)
    print("\nPresent all data in student table:")

    #Get all results,assign them to the list,fecthall() returns empty list if no results
    listOfResults=c.fetchall()

    # Save (commit) the changes
    conn.commit()

    # We close the connection and free all resources
    conn.close()
    
    # If given loginName and password exist in database then exactly one row should be returned.
    # We assume that loginNames are unique.
    if (len(listOfResults)==1):
        return True
    return False

@app.route('/login_page')
def login_page():
    return render_template('login.html')

@app.route('/login/', methods=['POST','GET'])
def login():
    loginName = request.form.get('login_name', default=False)
    password = request.form.get('password', default=False)

    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))

    if (CheckIfCorrectCredentials(loginName,password)==True):  
        return redirect(url_for('portfolioOverview'))
    else:
        return render_template('login.html')

@app.route('/register', methods=['POST','GET'])
def register():
    loginName = request.form.get('login_name', default=False)
    password1 = request.form.get('password1', default=False)
    password2 = request.form.get('password2', default=False)

    name = request.form.get('name', default=False)
    surname = request.form.get('surname', default=False)
    birthDate = request.form.get('birthDate', default=False)

    city = request.form.get('city', default=False)
    zipCode = request.form.get('zipCode', default=False)
    street = request.form.get('street', default=False)
    houseNo = request.form.get('houseNo', default=False)
    flatNo =request.form.get('flatNo', default=False)

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    c.execute("select userID from users order by userID desc limit 1")
    userID = c.fetchall()
    userID = userID[0][0]
    userID += 1
    
    c.execute("select personalInfoID from userPersonalInfo order by PersonalInfoID desc limit 1")
    personalInfoID = c.fetchall()
    personalInfoID = personalInfoID[0][0]
    personalInfoID += 1

    params1 = [loginName, password1]
    c.execute("insert into users(loginName, password) values (?, ?)", params1)

    params2 = [name, surname, birthDate, userID]
    c.execute("insert into userPersonalInfo(name, surname, birthDate, userID) values (?, ?, ?, ?)", params2)

    params3 = [city, zipCode, street, houseNo, flatNo, personalInfoID]
    c.execute("insert into userAddress(city, zipCode, street, houseNo, flatNo, personalInfoID) values (?, ?, ?, ?, ?, ?)", params3)

    conn.commit()
    conn.close()

    return render_template('portfolioOverview')

@app.route('/portfolioOverview')
def base():
    return render_template('portfolioOverview')

@app.route('/stockMarketInfo')
def stockMarketInfo():
    return render_template('stockMarketInfo')

@app.route('/portfolioEditor')
def portfolioEditor():
    return render_template('portfolioEditor')

@app.route('/personalData')
def personalData():
    return terender_templatemplate('personalData')

@app.route('/changePassword')
def changePassword():
    return render_template('changePassword')
